In [1]:
from text_gan.utils import SQuADReader
from text_gan import cfg

In [2]:
reader = SQuADReader()

In [3]:
filename = cfg.RAW_TRAIN_SAVE

In [4]:
parsed = reader.parse(filename)
filtered = reader.filter_unique_ca_pairs(parsed)
data = reader.flatten_parsed(filtered)

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
data[:5]

[{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'answer': 'Saint Bernadette Soubirous'},
 {'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately 

In [7]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[0]['question'])]

[('To', 'PART', 'TO'),
 ('whom', 'PRON', 'WP'),
 ('did', 'AUX', 'VBD'),
 ('the', 'DET', 'DT'),
 ('Virgin', 'PROPN', 'NNP'),
 ('Mary', 'PROPN', 'NNP'),
 ('allegedly', 'ADV', 'RB'),
 ('appear', 'VERB', 'VBP'),
 ('in', 'ADP', 'IN'),
 ('1858', 'NUM', 'CD'),
 ('in', 'ADP', 'IN'),
 ('Lourdes', 'PROPN', 'NNP'),
 ('France', 'PROPN', 'NNP'),
 ('?', 'PUNCT', '.')]

In [8]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[1]['question'])]

[('What', 'PRON', 'WP'),
 ('is', 'AUX', 'VBZ'),
 ('in', 'ADP', 'IN'),
 ('front', 'NOUN', 'NN'),
 ('of', 'ADP', 'IN'),
 ('the', 'DET', 'DT'),
 ('Notre', 'PROPN', 'NNP'),
 ('Dame', 'PROPN', 'NNP'),
 ('Main', 'PROPN', 'NNP'),
 ('Building', 'PROPN', 'NNP'),
 ('?', 'PUNCT', '.')]

In [9]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[2]['question'])]

[('The', 'DET', 'DT'),
 ('Basilica', 'PROPN', 'NNP'),
 ('of', 'ADP', 'IN'),
 ('the', 'DET', 'DT'),
 ('Sacred', 'ADJ', 'JJ'),
 ('heart', 'NOUN', 'NN'),
 ('at', 'ADP', 'IN'),
 ('Notre', 'PROPN', 'NNP'),
 ('Dame', 'PROPN', 'NNP'),
 ('is', 'AUX', 'VBZ'),
 ('beside', 'ADV', 'RB'),
 ('to', 'PART', 'TO'),
 ('which', 'DET', 'WDT'),
 ('structure', 'NOUN', 'NN'),
 ('?', 'PUNCT', '.')]

In [10]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[3]['question'])]

[('What', 'PRON', 'WP'),
 ('is', 'AUX', 'VBZ'),
 ('the', 'DET', 'DT'),
 ('Grotto', 'PROPN', 'NNP'),
 ('at', 'ADP', 'IN'),
 ('Notre', 'PROPN', 'NNP'),
 ('Dame', 'PROPN', 'NNP'),
 ('?', 'PUNCT', '.')]

In [11]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[4]['question'])]

[('What', 'PRON', 'WP'),
 ('sits', 'VERB', 'VBZ'),
 ('on', 'ADP', 'IN'),
 ('top', 'NOUN', 'NN'),
 ('of', 'ADP', 'IN'),
 ('the', 'DET', 'DT'),
 ('Main', 'PROPN', 'NNP'),
 ('Building', 'PROPN', 'NNP'),
 ('at', 'ADP', 'IN'),
 ('Notre', 'PROPN', 'NNP'),
 ('Dame', 'PROPN', 'NNP'),
 ('?', 'PUNCT', '.')]

In [12]:
[(token.text, token.pos_, token.tag_) for token in nlp(data[5]['question'])]

[('When', 'ADV', 'WRB'),
 ('did', 'AUX', 'VBD'),
 ('the', 'DET', 'DT'),
 ('Scholastic', 'PROPN', 'NNP'),
 ('Magazine', 'PROPN', 'NNP'),
 ('of', 'ADP', 'IN'),
 ('Notre', 'PROPN', 'NNP'),
 ('dame', 'NOUN', 'NN'),
 ('begin', 'VERB', 'VBP'),
 ('publishing', 'VERB', 'VBG'),
 ('?', 'PUNCT', '.')]

In [13]:
ques = map(lambda x: x['question'], data)
ques = nlp.pipe(ques, n_process=cfg.MAX_PARALLELISM)

In [14]:
has_vb = 0
has_noun = 0
noun = 'NOUN'
propn = 'PROPN'
verb = 'VERB'
length_till = []
length_after = []
for i, q in enumerate(ques):
    pos = [token.pos_ for token in q]
    if noun in pos:
        has_noun += 1
    elif propn in pos:
        has_noun += 1
    if verb in pos:
        has_vb += 1
        length_till.append(pos.index(verb) + 1)
        length_after.append(len(pos) - length_till[-1])
    else:
        posnoun = -1
        pospropn = -1
        try:
            posnoun = pos.index(noun)
        except:
            pass
        try:
            pospropn = pos.index(propn)
        except:
            pass
        idx = posnoun if posnoun != -1 and (pospropn == -1 or posnoun < pospropn) else pospropn
        if idx == -1:
            raise ValueError("weird")
        length_till.append(idx + 1)
        length_after.append(len(pos) - length_till[-1])

ValueError: weird

In [17]:
pos

['PRON', 'AUX', 'DET', 'ADJ', 'ADJ', 'PUNCT']

In [29]:
import numpy as np

In [59]:
np.percentile(np.array(length_till), 95)

10.0

In [54]:
data[1141]

{'context': 'In December 1901, Marconi successfully transmitted the letter S from England to Newfoundland, terminating Tesla\'s relationship with Morgan.[improper synthesis?] Over the next five years, Tesla wrote over 50 letters to Morgan, pleading for and demanding additional funding to complete the construction of Wardenclyffe. Tesla continued the project for another nine months. The tower was erected to its full 187 feet (57 m). In July 1903, Tesla wrote to Morgan that in addition to wireless communication, Wardenclyffe would be capable of wireless transmission of electric power. On 14 October 1904, Morgan finally replied through his secretary, stating, "It will be impossible for [me] to do anything in the matter," after Tesla had written to Morgan when the financier was meeting with the Archbishop of Canterbury in an attempt to appeal to his Christian spirit.',
 'question': 'What happened in 1901?',
 'answer': 'Marconi successfully transmitted the letter S from England to Newfoundl

In [61]:
has_vb

63858